In [105]:
import cv2
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
import heapq
import xlsxwriter
from xlutils.copy import copy    
from xlrd import open_workbook
import os
import math
from scipy import ndimage
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from hdf5datasetwriter import HDF5DatasetWriter
from imutils import paths
import numpy as np
import progressbar
import json
import cv2
import os
import random

In [66]:
# Search all data with _highBW.mat and save to files
path = os.path.abspath("/media/peppermint/easystore/123/Patients")
print(path)
files = []
paths=[];
# Find file with _ute.ni 

for r, d, f in os.walk(path):
    d.sort()
    for file in f:
        if '.mat' in file:
            index=-3;
            name=[];
            find_=False;
            #print(r)
            while(not find_):
                if(file[index]=='.'):
                    find_= True;
                index=index-1;
                if(find_):
                    name.append(file[0:index+1]);
            files.append(name[0])
            paths.append(r)


/media/peppermint/easystore/123/Patients


In [67]:
files

['000001',
 '000001_highBW',
 '000001A',
 '000001A_highBW',
 '000001B_highBW',
 '000001H_highBW',
 '000001P_highBW',
 '001001_highBW',
 '002011_highBW',
 '002011_hrep',
 '002071A',
 '002071A_rep',
 '002077A',
 '002077A_rep',
 '002087A',
 '002087A_rep',
 '002087B',
 '002087B_highBW',
 '002087C_highBW',
 '002094C',
 '002094C_highBW',
 '002098',
 '002098A',
 '002099',
 '002099_rep',
 '002100',
 '002101',
 '002101_highBW',
 '002102',
 '002103',
 '002104',
 '002105',
 '002105_highBW',
 '002106',
 '002106_highBW',
 '002107',
 '002108',
 '002108_highBW',
 '002109_highBW',
 '002110',
 '002110_highBW',
 '002111',
 '002111_highBW',
 '002112_highBW',
 '002113_highBW',
 '002114_highBW',
 '002115_highBW',
 '002116',
 '002116_highBW',
 '002116_hrep',
 '002116_rep',
 '002117_highBW',
 '002118_highBW',
 '002119_highBW',
 '002120_highBW',
 '002121_highBW',
 '002122_highBW',
 '002123_highBW',
 '002124_highBW',
 '002125_highBW',
 '002126_highBW',
 '002127_highBW',
 '002128_highBW',
 '002129_highBW',
 '00

In [68]:
len(files)

352

In [69]:
len(paths)

352

In [70]:
files_df = pd.DataFrame({'file':files, 'path':paths})

In [71]:
files_df.to_csv('456.csv')

In [122]:
label = pd.read_csv('label.csv');
label = label.drop('Unnamed: 0',axis=1);
label = label.rename(columns={"0": "file", "0.1": "label"})

In [123]:
label.shape

(91, 2)

In [124]:
label = pd.merge(label,files_df,how = 'left',on=['file'])
label.shape

(91, 3)

In [125]:
label = label.drop_duplicates(subset = 'file' )
label.shape

(91, 3)

In [126]:
label.head()

,file,label,path
0,003023_highBW,IPF,/media/peppermint/easystore/123/Patients/003-0...
1,003023_hrep,IPF,/media/peppermint/easystore/123/Patients/003-0...
2,002112_highBW,NSIP,/media/peppermint/easystore/123/Patients/002-1...
3,002113_highBW,NSIP,/media/peppermint/easystore/123/Patients/002-1...
4,002114_highBW,NSIP,/media/peppermint/easystore/123/Patients/002-1...


In [82]:
label.iloc[0]['file']

'003023_highBW'

In [127]:
label.shape

(91, 3)

In [109]:
shuffle_index = np.arange(len(label))
np.random.shuffle(shuffle_index)
print(shuffle_index)

[70 28 14 21  3 87 61 76 66 15 63 81 19 24 55  8 60 18 54 62 50 44 13 20
 40 82 26 80 67 85  6 39  4 73 75 65 47 72 10 42 83 17  9 58 74 25 68 37
 86 34 77  5  7 90 43 36 12 46 89 45 22 38  1  2 53 52 35 48 69  0 31 56
 11 27 32 64 78 30 88 41 49 79 33 71 84 23 59 51 16 57 29]


In [110]:
shuffle_index[1]

28

In [119]:
#label = label.iloc[0:2]

In [128]:


im_width = 128
im_height = 128
slices = 10;
border = 5

shuffle_index = np.arange(len(label))
np.random.shuffle(shuffle_index)

outputPath='bar_rbc_gas_10.hdf5'

writer = HDF5DatasetWriter((len(label),slices, im_width, im_height, 3),(len(label), 1), outputPath)


for _index in range(len(label)):

    num_file = shuffle_index[_index];
    
    current_path = str(label.iloc[num_file]['path'])+'/'+str(label.iloc[num_file]['file'])+'.mat';

    label_str_current = label.iloc[num_file]['label']
    
    if(label_str_current=='IPF'):
        label_current = 0;
    elif(label_str_current=='NSIP'):
        label_current = 1;
    print(label_str_current)
    print(label_current)
        
        

    mat = scipy.io.loadmat(current_path)

    type_of_data=['bar2gas', 'rbc2gas','gas_highreso_cor'];

    number_types=len(type_of_data);

    mask_reg = mat['mask_reg']

    [rx,ry,rz]= np.array(mask_reg).shape;

    print(_index)
    print(num_file)
    print('Now Doing')
    print(label.iloc[num_file]['file'])
    

    total_mean=[];
    ij = 0;

    while(sum(sum(mask_reg[:,:,ij]))<900):

        ij = ij+1;

    first_index = ij;

    ij = 127;
    
    print(first_index)

    while(sum(sum(mask_reg[:,:,ij]))<900):

        ij = ij-1;

    last_index = ij;
    
    #print(last_index)
    ten_slices = np.round(np.arange(first_index,last_index,(last_index-first_index)/10.0)).astype(int)
    #print(ten_slices)
    
    data_images=np.zeros([rx,ry,10,number_types]);
    
    for num,wtype in enumerate(type_of_data): 
        data_images[:,:,:,num]= mat[wtype][:,:,ten_slices];
        #print(mat[wtype].shape)
        #print(mat[wtype][:,:,ten_slices].shape)

    data_images = data_images.reshape((10,128,128,3))
    print(data_images.shape)
    
    writer.add([data_images], [[label_current]])

writer.close()


NSIP
1
0
7
Now Doing
002118_highBW
34
(10, 128, 128, 3)
NSIP
1
1
30
Now Doing
002150_highBW
50
(10, 128, 128, 3)
IPF
0
2
86
Now Doing
003024A_hrep
42
(10, 128, 128, 3)
IPF
0
3
89
Now Doing
003025A_highBW
42
(10, 128, 128, 3)
IPF
0
4
55
Now Doing
003016_hrep
44
(10, 128, 128, 3)
IPF
0
5
81
Now Doing
003022A_highBW
44
(10, 128, 128, 3)
IPF
0
6
67
Now Doing
003018A_highBW
41
(10, 128, 128, 3)
NSIP
1
7
37
Now Doing
002164_highBW
31
(10, 128, 128, 3)
IPF
0
8
57
Now Doing
003016A_hrep
46
(10, 128, 128, 3)
IPF
0
9
62
Now Doing
003017A_highBW
27
(10, 128, 128, 3)
NSIP
1
10
5
Now Doing
002115_highBW
44
(10, 128, 128, 3)
NSIP
1
11
27
Now Doing
002146_highBW
58
(10, 128, 128, 3)
IPF
0
12
61
Now Doing
003017_hrep
26
(10, 128, 128, 3)
IPF
0
13
42
Now Doing
003013B
30
(10, 128, 128, 3)
NSIP
1
14
3
Now Doing
002113_highBW
51
(10, 128, 128, 3)
NSIP
1
15
28
Now Doing
002148_highBW
33
(10, 128, 128, 3)
NSIP
1
16
38
Now Doing
002165_highBW
28
(10, 128, 128, 3)
IPF
0
17
73
Now Doing
003019B_highBW
45
(10,